In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import gc 
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')
#最大表示行数の指定（ここでは50行を指定）
pd.set_option('display.max_rows', 1000)
#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 1000)
#infoの省略を阻止
pd.options.display.max_info_rows=500
pd.options.display.max_info_columns=500

import pandas_profiling as pdp
import sklearn as skl
import lightgbm as lgb

# elite_num = importance[:24].index
# elite = ["f_231", "f_74", "f_153", "f_225", "f_165", "f_145", "f_62", "f_108", "f_22", "f_142", "f_63", "target"]

In [ ]:
elite = ["f_231", "f_74", "f_153", "f_225", "f_165", "f_145", "f_62", "f_108", "f_22", "f_142", "f_63", ]

In [ ]:
# data = pd.read_csv("../input/ubiquant-market-prediction/train.csv", usecols=elite)
df_train = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')

In [ ]:
# gm = ["f_245", "f_205", "f_266","f_258", "f_43", "target"]

In [ ]:
# gmdata = pd.read_csv("../input/ubiquant-market-prediction/train.csv", usecols=gm)

In [ ]:
# gmdata.profile_report()

gmdata.profile_report()

In [ ]:
# x = data.drop("target", axis=1)
# y = data["target"]

# from sklearn.model_selection import train_test_split

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)


In [ ]:
x = df_train.drop(["row_id", "time_id", "target"], axis=1)
x = x[elite]
y = df_train["target"]

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [ ]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_test = lgb.Dataset(x_test, y_test)
params = {'objective':'regression'}

In [ ]:
def cor(preds, eval_data):
    eval_data = eval_data.get_label()
    eval_data = eval_data.astype("float64")
    preds = np.array(preds)
    eval_data = np.array(eval_data)
    res = np.corrcoef(preds, eval_data)
    return "cor", res[0][1], True

In [ ]:
model = lgb.train(params, lgb_train, 
                  valid_sets=[lgb_train,lgb_test], 
                  num_boost_round=1000, early_stopping_rounds=100, 
                 feval=cor)
y_pred = model.predict(x_test, num_iteration=model.best_iteration)


In [ ]:
# importance = pd.DataFrame(model.feature_importance(importance_type = 'gain'),
#                            index=x.columns, columns=['importance'])
# importance = importance.sort_values('importance', ascending=False)
# # display(importance)
# plt.hist(importance)

In [ ]:
#readlist = [267, 208, 149, 71, 233, 77, 187, 82]
# elite_data = pd.read_csv("../input/ubiquant-market-prediction/train.csv", usecols=elite_num)

In [ ]:
# df_train = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')

In [ ]:
# pdp.ProfileReport(df_train)

In [ ]:
def predictLGB(prediction_df):
    X_predict = prediction_df.drop(['row_id'],axis=1)
    X_predict = X_predict[elite]
    prediction = model.predict(X_predict)
    return prediction

In [ ]:
import ubiquant

env = ubiquant.make_env()  # initialize the environment
iter_test = env.iter_test()  # an iterator which loops over the test set and sample submission

#テストデータから不要列落とす

for (test_df, sample_prediction_df) in iter_test:
    # test_df.info()
    sample_prediction_df['target'] = predictLGB(test_df) # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions